analise de dados de vendas de supermercados de curitiba

In [26]:
import pandas as pd

df = pd.read_csv("./files/economia_curitiba.csv", sep=";", encoding="latin-1")
pd.set_option('display.max_columns', None)
df.head()

,data_pesquisa,id_empresa,rede,endereco_completo,codigo_categoria,id_produto,descricao,preco_regular,preco_atacado,preco_atacado_qtd,preco_promocao,preco_fidelidade
0,2025-07-31,17.0,Agricer,"Rua Izaac Ferreira da Cruz, 1440 - Pinheirinho...",5.0,2048,BACON SUÍNO -FRIGONOVAK -1 KG,34.99,0.0,0,0.0,0.0
1,2025-07-31,17.0,Agricer,"Rua Izaac Ferreira da Cruz, 1440 - Pinheirinho...",5.0,2055,LINGUIÇA CALABRESA -FRIGONOVAK -1 KG,34.99,0.0,0,0.0,0.0
2,2025-07-31,17.0,Agricer,"Rua Izaac Ferreira da Cruz, 1440 - Pinheirinho...",1.0,7891000352175,ACHOCOLATADO EM PÓ POTE -NESCAU -370 GR,8.75,0.0,0,0.0,0.0
3,2025-07-31,17.0,Agricer,"Rua Izaac Ferreira da Cruz, 1440 - Pinheirinho...",1.0,7891962015620,BOLINHO RECHEADO SABORES -BAUDUCCO -40 GR,2.69,0.0,0,0.0,0.0
4,2025-07-31,17.0,Agricer,"Rua Izaac Ferreira da Cruz, 1440 - Pinheirinho...",1.0,7896016700149,CAFÉ A VÁCUO EXTRA FORTE -ALVORADA -500 GR,26.99,0.0,0,0.0,0.0


informações e tipos de colunas.

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13592 entries, 0 to 13591
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   data_pesquisa      13592 non-null  object 
 1   id_empresa         12240 non-null  float64
 2   rede               13592 non-null  object 
 3   endereco_completo  13592 non-null  object 
 4   codigo_categoria   13296 non-null  float64
 5   id_produto         13592 non-null  int64  
 6   descricao          13592 non-null  object 
 7   preco_regular      13592 non-null  float64
 8   preco_atacado      13592 non-null  float64
 9   preco_atacado_qtd  13592 non-null  int64  
 10  preco_promocao     13592 non-null  float64
 11  preco_fidelidade   13592 non-null  float64
dtypes: float64(6), int64(2), object(4)
memory usage: 1.2+ MB


removendo aspas, espaços e convertendo a coluna para datetime.

In [28]:
df['data_pesquisa'] = pd.to_datetime(df['data_pesquisa'].astype(str).str.replace("'", "").str.strip(), format="%Y-%m-%d")

convertendo coluna id_empresa para numeros inteiros.

In [29]:
df['id_empresa'] = df['id_empresa'].astype('Int64')

contagem de linhas nulas

In [30]:
df.isnull().sum()

data_pesquisa           0
id_empresa           1352
rede                    0
endereco_completo       0
codigo_categoria      296
id_produto              0
descricao               0
preco_regular           0
preco_atacado           0
preco_atacado_qtd       0
preco_promocao          0
preco_fidelidade        0
dtype: int64

porcentagem das linhas vazias, de acordo com esse dado podemos tomar a decisão de excluir ou manter.

In [31]:
def porcentagem_nulos(value):
    return f"{100 * value:.2f}%"
(df.isna().sum() / len(df)).sort_values(ascending=False).apply(porcentagem_nulos)

id_empresa           9.95%
codigo_categoria     2.18%
rede                 0.00%
data_pesquisa        0.00%
endereco_completo    0.00%
id_produto           0.00%
descricao            0.00%
preco_regular        0.00%
preco_atacado        0.00%
preco_atacado_qtd    0.00%
preco_promocao       0.00%
preco_fidelidade     0.00%
dtype: object

In [32]:
df["rede"].value_counts()

rede
Condor                            2265
Atacadão                          1345
Carrefour                          974
Festval                            904
Tissi                              628
Jacomar                            609
Angeloni                           561
Super Muffato                      554
Nacional                           554
Max Atacadista                     487
Zamprogna                          472
Michel                             464
Tok Super                          443
Italo                              428
Super Boni                         419
Stall                              407
Palmeira                           388
Agricer                            369
Mini Preço                         345
Araucária                          340
Triunfo                            327
Telêmaco                            86
Bonfim                              81
Hipermercado Condor Novo Mundo      56
Boni Bacacheri                      33
Casa Fiesta         

identificando nomes de mercados iguais com nomes diferentes

In [33]:
df.loc[df["rede"].str.contains("condor", case=False, na=False), "rede"] = "condor"
df.loc[df["rede"].str.contains("JUMBO ATACADISTA", case=False, na=False), "rede"] = "jumbo atacadista"

In [34]:
df["bairro"] = df["endereco_completo"].str.extract(r'- ([^-\n]+) - Curitiba/PR')
df.loc[df["endereco_completo"].str.contains("Rua Eduardo Sprada", case=False, na=False), "bairro"] = "Campo Comprido"
df.loc[df["endereco_completo"].str.contains("Avenida Ver Toaldo", case=False, na=False), "bairro"] = "São Braz"
df.loc[df["endereco_completo"].str.contains("Rua Izaac Ferreira", case=False, na=False), "bairro"] = "Sítio Cercado"
df.loc[df["endereco_completo"].str.contains("Rua João Alencar", case=False, na=False), "bairro"] = "Santa Quitéria"

In [35]:
# Exemplo de mapeamento (ajuste conforme seu dicionário real)
categoria_map = {
    1.0: "Alimentos",
    2.0: "Bebidas",
    3.0: "Higiene",
    4.0: "Padaria",
    5.0: "Açougue",
    6.0: "Limpeza",
    7.0: "Utilidades",
    8.0: "Hortifruti",
    9.0: "Bebidas alcoólicas"
}

# Cria a coluna de categoria
df["categoria"] = df["codigo_categoria"].map(categoria_map)

# Calcula a variação de preço (desvio padrão) por categoria
variacao_categoria = df.groupby("categoria")["preco_regular"].std().sort_values(ascending=False)

display(variacao_categoria)

categoria
Açougue               16.912532
Bebidas alcoólicas    15.782591
Higiene               11.102241
Alimentos              9.740003
Limpeza                7.201481
Hortifruti             6.874706
Utilidades             6.287779
Bebidas                4.191503
Padaria                2.926909
Name: preco_regular, dtype: float64

In [36]:
df.head()

,data_pesquisa,id_empresa,rede,endereco_completo,codigo_categoria,id_produto,descricao,preco_regular,preco_atacado,preco_atacado_qtd,preco_promocao,preco_fidelidade,bairro,categoria
0,2025-07-31,17,Agricer,"Rua Izaac Ferreira da Cruz, 1440 - Pinheirinho...",5.0,2048,BACON SUÍNO -FRIGONOVAK -1 KG,34.99,0.0,0,0.0,0.0,Sítio Cercado,Açougue
1,2025-07-31,17,Agricer,"Rua Izaac Ferreira da Cruz, 1440 - Pinheirinho...",5.0,2055,LINGUIÇA CALABRESA -FRIGONOVAK -1 KG,34.99,0.0,0,0.0,0.0,Sítio Cercado,Açougue
2,2025-07-31,17,Agricer,"Rua Izaac Ferreira da Cruz, 1440 - Pinheirinho...",1.0,7891000352175,ACHOCOLATADO EM PÓ POTE -NESCAU -370 GR,8.75,0.0,0,0.0,0.0,Sítio Cercado,Alimentos
3,2025-07-31,17,Agricer,"Rua Izaac Ferreira da Cruz, 1440 - Pinheirinho...",1.0,7891962015620,BOLINHO RECHEADO SABORES -BAUDUCCO -40 GR,2.69,0.0,0,0.0,0.0,Sítio Cercado,Alimentos
4,2025-07-31,17,Agricer,"Rua Izaac Ferreira da Cruz, 1440 - Pinheirinho...",1.0,7896016700149,CAFÉ A VÁCUO EXTRA FORTE -ALVORADA -500 GR,26.99,0.0,0,0.0,0.0,Sítio Cercado,Alimentos


In [ ]:
# df2 = df.to_csv("df_format.csv", index=False)